<a href="https://colab.research.google.com/github/ngolemai020405/Thuchanh_TTNT/blob/main/BaoCao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import math
from heapq import heappush, heappop

def read_int(prompt, lo=None, hi=None):
    while True:
        try:
            x = int(input(prompt).strip())
            if lo is not None and x < lo:
                print(f"Giá trị phải >= {lo}")
                continue
            if hi is not None and x > hi:
                print(f"Giá trị phải <= {hi}")
                continue
            return x
        except ValueError:
            print("Nhập số nguyên hợp lệ.")

def read_choice(prompt, options):
    opts = [o.lower() for o in options]
    while True:
        s = input(prompt).strip().lower()
        if s in opts:
            return s
        print(f"Chọn 1 trong: {', '.join(options)}")

DIRS = [(1,0),(0,-1),(-1,0),(0,1)]

class PuzzleNode:
    def __init__(self, parent, mats, empty_pos, h, level):
        self.parent = parent
        self.mats = mats
        self.empty_pos = empty_pos
        self.h = h
        self.level = level
    def __lt__(self, other):
        return (self.h + self.level) < (other.h + other.level)

def puzzle_read_matrix(n, title):
    print(f"\nNhập ma trận {title} ({n} dòng, mỗi dòng {n} số, 0 là ô trống):")
    mats = []
    for i in range(n):
        while True:
            parts = input(f"Row {i+1}: ").strip().split()
            if len(parts) != n:
                print(f"Phải nhập đúng {n} số.")
                continue
            try:
                row = [int(p) for p in parts]
            except ValueError:
                print("Chỉ nhập số nguyên.")
                continue
            mats.append(row)
            break
    return mats

def puzzle_find_zero(mats):
    for i,row in enumerate(mats):
        for j,v in enumerate(row):
            if v == 0:
                return (i,j)
    return None

def puzzle_validate(initial, goal):
    a = sorted([x for r in initial for x in r])
    b = sorted([x for r in goal for x in r])
    return a == b and len(a) == len(set(a)) and a[0] == 0

def puzzle_h(curr, goal):
    n = len(curr)
    wrong = 0
    for i in range(n):
        for j in range(n):
            if curr[i][j] != 0 and curr[i][j] != goal[i][j]:
                wrong += 1
    return wrong

def puzzle_print(mats):
    for r in mats:
        print(" ".join(f"{v:2d}" for v in r))

def puzzle_print_path(node):
    path = []
    while node:
        path.append(node.mats)
        node = node.parent
    path.reverse()
    for step, mats in enumerate(path):
        print(f"\nStep {step}:")
        puzzle_print(mats)

def run_puzzle():
    print("\n=== DEMO 1: AKT / N-PUZZLE ===")
    n = read_int("Nhập kích thước n (3=8puzzle, 4=15puzzle) [2..5]: ", lo=2, hi=5)
    initial = puzzle_read_matrix(n, "INITIAL")
    goal = puzzle_read_matrix(n, "GOAL")

    if not puzzle_validate(initial, goal):
        print("❌ Initial/Goal không hợp lệ (phải cùng tập giá trị, không trùng, có 0).")
        return

    limit = read_int("Giới hạn số trạng thái duyệt (để tránh chạy lâu) [1000..50000]: ", lo=1000, hi=50000)

    empty = puzzle_find_zero(initial)
    if empty is None:
        print("❌ Không tìm thấy ô trống (0).")
        return

    root = PuzzleNode(None, initial, empty, puzzle_h(initial, goal), 0)
    pq = []
    heappush(pq, root)

    visited = set()
    visited.add(tuple(map(tuple, initial)))

    expanded = 0
    while pq and expanded < limit:
        cur = heappop(pq)
        expanded += 1

        if cur.h == 0:
            print("\n✅ Giải pháp tìm thấy!")
            print(f"Số bước: {cur.level} | Số trạng thái đã duyệt: {expanded}")
            puzzle_print_path(cur)
            return

        x,y = cur.empty_pos
        for dx,dy in DIRS:
            nx, ny = x+dx, y+dy
            if 0 <= nx < n and 0 <= ny < n:
                new_m = copy.deepcopy(cur.mats)
                new_m[x][y], new_m[nx][ny] = new_m[nx][ny], new_m[x][y]
                key = tuple(map(tuple, new_m))
                if key in visited:
                    continue
                visited.add(key)
                child = PuzzleNode(cur, new_m, (nx,ny), puzzle_h(new_m, goal), cur.level+1)
                heappush(pq, child)

    print("⚠️ Không tìm thấy lời giải trong giới hạn duyệt. (Hoặc case khó / heuristic yếu)")


def astar_load_edges(path):
    adj = {}
    nodes = set()
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            u,v,w = line.split()
            w = float(w)
            nodes.add(u); nodes.add(v)
            adj.setdefault(u, []).append((v,w))
            # undirected by default (dễ demo)
            adj.setdefault(v, []).append((u,w))
    for n in nodes:
        adj.setdefault(n, [])
    return adj

def astar_run(adj, h, start, goal):
    open_set = {start}
    closed = set()
    g = {start: 0.0}
    parent = {start: None}

    def f(n): return g.get(n, math.inf) + h.get(n, 10**9)

    while open_set:
        n = min(open_set, key=f)
        if n == goal:
            path = []
            cur = n
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path.reverse()
            return path, g[n]

        open_set.remove(n)
        closed.add(n)

        for m,w in adj.get(n, []):
            if m in closed:
                continue
            tentative = g[n] + w
            if m not in open_set:
                open_set.add(m)
            if tentative < g.get(m, math.inf):
                g[m] = tentative
                parent[m] = n

    return None, math.inf

def run_astar():
    print("\n=== DEMO 2: A* ===")
    mode = read_choice("Chọn dữ liệu graph (default/file): ", ["default", "file"])

    if mode == "default":
        adj = {
            "A": [("B", 1), ("C", 2)],
            "B": [("A", 1), ("D", 5)],
            "C": [("A", 2), ("D", 1)],
            "D": [("B", 5), ("C", 1)],
        }
        nodes = sorted(adj.keys())
        print("Nodes:", nodes)
    else:
        path = input("Nhập đường dẫn file edges (u v w mỗi dòng): ").strip()
        try:
            adj = astar_load_edges(path)
        except Exception as e:
            print("❌ Lỗi đọc file:", e)
            return
        nodes = sorted(adj.keys())
        print("Nodes:", nodes)

    h = {}
    use_h = read_choice("Heuristic (all1/manual): ", ["all1", "manual"])
    if use_h == "all1":
        for n in nodes:
            h[n] = 1
    else:
        print("Nhập heuristic h(n) cho từng node (số nguyên/số thực):")
        for n in nodes:
            while True:
                try:
                    h[n] = float(input(f"h({n}) = ").strip())
                    break
                except ValueError:
                    print("Nhập số hợp lệ.")

    start = input("Chọn điểm bắt đầu: ").strip()
    goal = input("Chọn điểm kết thúc: ").strip()
    if start not in adj or goal not in adj:
        print("❌ Start/Goal không tồn tại trong graph.")
        return

    path, cost = astar_run(adj, h, start, goal)
    if path is None:
        print("❌ Không tìm thấy đường đi.")
    else:
        print("✅ Path:", " -> ".join(path))
        print("✅ Cost:", cost)


EMPTY = None

def ttt_init(n):
    return [[EMPTY]*n for _ in range(n)]

def ttt_actions(board):
    n = len(board)
    return [(i,j) for i in range(n) for j in range(n) if board[i][j] is EMPTY]

def ttt_player(board):
    cnt = sum(cell is not EMPTY for row in board for cell in row)
    return "X" if cnt % 2 == 0 else "O"

def ttt_winner(board):
    n = len(board)
    lines = []
    lines.extend(board)
    lines.extend([list(col) for col in zip(*board)])
    lines.append([board[i][i] for i in range(n)])
    lines.append([board[i][n-1-i] for i in range(n)])

    for line in lines:
        if line[0] is not EMPTY and all(c == line[0] for c in line):
            return line[0]
    return None

def ttt_terminal(board):
    return ttt_winner(board) is not None or len(ttt_actions(board)) == 0

def ttt_utility(board):
    w = ttt_winner(board)
    if w == "X": return 1
    if w == "O": return -1
    return 0

def ttt_result(board, action):
    i,j = action
    if board[i][j] is not EMPTY:
        raise ValueError("Ô đã có quân.")
    nb = copy.deepcopy(board)
    nb[i][j] = ttt_player(board)
    return nb

def ttt_print(board):
    n = len(board)
    for i in range(n):
        row = [(board[i][j] if board[i][j] is not None else ".") for j in range(n)]
        print(" ".join(row))
    print()

def ttt_heuristic(board):
    n = len(board)
    score = 0

    def line_score(line):
        nonlocal score
        if all(c in (EMPTY, "X") for c in line):
            score += 1
        if all(c in (EMPTY, "O") for c in line):
            score -= 1

    for r in board: line_score(r)
    for c in zip(*board): line_score(list(c))
    line_score([board[i][i] for i in range(n)])
    line_score([board[i][n-1-i] for i in range(n)])
    return score / (2*n + 2)

def minimax(board, depth_limit=None):
    cur = ttt_player(board)

    def maxv(state, depth):
        if ttt_terminal(state):
            return ttt_utility(state), None
        if depth_limit is not None and depth >= depth_limit:
            return ttt_heuristic(state), None
        best = (-math.inf, None)
        for a in ttt_actions(state):
            val, _ = minv(ttt_result(state, a), depth+1)
            if val > best[0]:
                best = (val, a)
        return best

    def minv(state, depth):
        if ttt_terminal(state):
            return ttt_utility(state), None
        if depth_limit is not None and depth >= depth_limit:
            return ttt_heuristic(state), None
        best = (math.inf, None)
        for a in ttt_actions(state):
            val, _ = maxv(ttt_result(state, a), depth+1)
            if val < best[0]:
                best = (val, a)
        return best

    if cur == "X":
        _, move = maxv(board, 0)
    else:
        _, move = minv(board, 0)
    return move

def run_minimax():
    print("\n=== DEMO 3: MINIMAX (CARO/TIC-TAC-TOE) ===")
    n = read_int("Nhập kích thước bàn cờ n [3..5]: ", lo=3, hi=5)
    board = ttt_init(n)

    human = input("Bạn chọn quân (X/O): ").strip().upper()
    if human not in ("X","O"):
        human = "X"
    ai = "O" if human == "X" else "X"

    depth_limit = None if n == 3 else 3
    if depth_limit is not None:
        print(f"(Note) n={n} dùng depth_limit={depth_limit} để chạy nhanh khi demo.")

    while True:
        ttt_print(board)
        if ttt_terminal(board):
            w = ttt_winner(board)
            if w is None:
                print("Kết thúc: HÒA")
            else:
                print(f"Kết thúc: {w} THẮNG")
            return

        cur = ttt_player(board)
        if cur == ai:
            mv = minimax(board, depth_limit=depth_limit)
            if mv is None:
                continue
            print(f"AI đi: row={mv[0]+1}, col={mv[1]+1}")
            board = ttt_result(board, mv)
        else:
            r = read_int("Row: ", lo=1, hi=n) - 1
            c = read_int("Col: ", lo=1, hi=n) - 1
            if (r,c) not in ttt_actions(board):
                print("Ô không hợp lệ/đã có quân. Nhập lại.")
                continue
            board = ttt_result(board, (r,c))

def main():
    while True:
        print("\n==============================")
        print(" DEMO 3 BÀI (AKT / A* / MINIMAX)")
        print("==============================")
        print("1) AKT / N-Puzzle (8/15 puzzle)")
        print("2) A* (graph pathfinding)")
        print("3) Minimax (caro/tic-tac-toe)")
        print("0) Thoát")
        ch = read_int("Chọn: ", lo=0, hi=3)
        if ch == 0:
            break
        if ch == 1:
            run_puzzle()
        elif ch == 2:
            run_astar()
        elif ch == 3:
            run_minimax()

if __name__ == "__main__":
    main()
21


 DEMO 3 BÀI (AKT / A* / MINIMAX)
1) AKT / N-Puzzle (8/15 puzzle)
2) A* (graph pathfinding)
3) Minimax (caro/tic-tac-toe)
0) Thoát
Chọn: 2

=== DEMO 2: A* ===
Chọn dữ liệu graph (default/file): default
Nodes: ['A', 'B', 'C', 'D']
Heuristic (all1/manual): A
Chọn 1 trong: all1, manual
Heuristic (all1/manual): all1
Chọn điểm bắt đầu: 2
Chọn điểm kết thúc: 5
❌ Start/Goal không tồn tại trong graph.

 DEMO 3 BÀI (AKT / A* / MINIMAX)
1) AKT / N-Puzzle (8/15 puzzle)
2) A* (graph pathfinding)
3) Minimax (caro/tic-tac-toe)
0) Thoát
Chọn: 3

=== DEMO 3: MINIMAX (CARO/TIC-TAC-TOE) ===
Nhập kích thước bàn cờ n [3..5]: 3
Bạn chọn quân (X/O): X
. . .
. . .
. . .

Row: 1
Col: 2
. X .
. . .
. . .

AI đi: row=1, col=1
O X .
. . .
. . .

Row: 2
Col: 2
O X .
. X .
. . .

AI đi: row=3, col=2
O X .
. X .
. O .

Row: 3
Col: 21
Giá trị phải <= 3
Col: 1
O X .
. X .
X O .

AI đi: row=1, col=3
O X O
. X .
X O .

Row: 2
Col: 3
O X O
. X X
X O .

AI đi: row=2, col=1
O X O
O X X
X O .

